# HSE 2021: Mathematical Methods for Data Analysis

## Homework 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import datasets
from sklearn.datasets import load_boston
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLSResults
from math import sqrt
import random
import sys

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

sns.set(style="darkgrid")

### Data

For this homework we use Dataset from seaborn on diamonds prices.

In [ ]:
data = sns.load_dataset('diamonds')

y = data.price
X = data.drop(['price'], axis=1)
columns = data.drop(['price'], axis=1).columns

## Linear regression

#### 0. [0.25 points] Encode categorical variables.

In [ ]:
# your code here
to_encode = ["cut", "color", "clarity"]

encoding = dict()
for word in to_encode:
    encoding[word] = dict()
    for index, elem in enumerate(data[word].unique()):
        encoding[word][elem] = index
X = X.replace(encoding)

#### 1. [0.25 points] Split the data into train and test sets with ratio 80:20 with random_state=17.

In [ ]:
# your code here 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

#### 2. [1 point] Train models on train data using StatsModels library and apply it to the test set; use $RMSE$ and $R^2$ as the quality measure.

* [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html);
* [`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) with $\alpha = 0.01$;
* [`Lasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) with $\alpha = 0.01$
* [`ElasticNet`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) with $\alpha = 0.01$, $l_{1}$_$ratio = 0.6$

Don't forget to scale the data before training the models with StandardScaler!

In [ ]:
# your code here 
from sklearn.preprocessing import StandardScaler

def print_quality(y_test, prediction, type_str):
    print(f'[{type_str}] RMSE = {mean_squared_error(y_test, prediction, squared = False)}')
    print(f'[{type_str}] R2   = {r2_score(y_test, prediction)}')
    print()

def get_fit(model, type_str):
    if type_str == "Linear":
        return model.fit()
    elif type_str == "Ridge":
        return model.fit_regularized(alpha = 0.01, L1_wt = 2 * sys.float_info.epsilon)
    elif type_str == "Lasso":
        return model.fit_regularized(alpha = 0.01, L1_wt = 1)
    elif type_str == "ElasticNet":
        return model.fit_regularized(alpha = 0.01, L1_wt = 0.6)

scaler = StandardScaler()
X_train_scaled = sm.add_constant(scaler.fit_transform(X_train))
X_test_scaled = sm.add_constant(scaler.transform(X_test))

types = ["Linear", "Ridge", "Lasso", "ElasticNet"]
for type_str in types:
    model = sm.OLS(y_train, X_train_scaled)
    fit = get_fit(model, type_str)
    prediction = fit.predict(X_test_scaled)
    print_quality(y_test, prediction, type_str)

[Linear] RMSE = 1448.264793807763
[Linear] R2   = 0.8704754690827957

[Ridge] RMSE = 1456.7092504515917
[Ridge] R2   = 0.8689606176297138

[Lasso] RMSE = 1446.990447222547
[Lasso] R2   = 0.8707033093583628

[ElasticNet] RMSE = 1448.229656468968
[ElasticNet] R2   = 0.8704817539723205



#### 3. [1 point] Explore the values of the parameters of the resulting models and compare the number of zero weights in them. Comment on the significance of the coefficients, overal model significance and other related factors from the results table

In [ ]:
# your code here
for type_str in types:
    model = sm.OLS(y_train, X_train_scaled)
    fit = get_fit(model, type_str)
    print(type_str, str(fit.params).replace("\n", " | ").replace("       ", " = "))

model = sm.OLS(y_train, X_train_scaled)
fit = get_fit(model, "Linear")
print("\nLinear", fit.summary(), "\n" * 2, "=" * 60, "\n" * 2)

Linear const    3928.681289 | x1 = 4970.821393 | x2 = -100.623988 | x3 =  181.037242 | x4 =  452.812052 | x5 = -227.657744 | x6 = -133.591532 | x7      -1273.901094 | x8 =   78.114669 | x9 =   50.601895 | dtype: float64
Ridge const    3889.783455 | x1 = 3978.162860 | x2 = -101.146981 | x3 =  167.583345 | x4 =  470.756273 | x5 = -166.061617 | x6 = -124.026129 | x7 = -252.063228 | x8 =   66.910711 | x9 =   14.034773 | dtype: float64
Lasso const    3928.671289 | x1 = 4787.133672 | x2 = -100.822264 | x3 =  179.157127 | x4 =  459.470274 | x5 = -215.429269 | x6 = -133.337373 | x7      -1086.947195 | x8 =   88.844953 | x9 =   34.960575 | dtype: float64
ElasticNet const    3913.023197 | x1 = 4423.366780 | x2 = -100.676516 | x3 =  173.980640 | x4 =  464.928746 | x5 = -191.804274 | x6 = -129.593753 | x7 = -674.929709 | x8 =   56.924328 | x9 =   11.092608 | dtype: float64

Linear                             OLS Regression Results                            
Dep. Variable:                  price  

1) У всех четырех моделей кол-во нулевых (близких к нулю) коэффициентов равно нулю :) Нет нулевых коэффициентов.

2) Рассмотри таблицу результатов базовой линейной модели. Общее количество наблюдений равно 43’152 c 9 степенями свободы в модели. В целом модель получилось значимой, так как Prob (F-statistic) = 0.00. Значение R-squared, которое равно 0.87, показывает какую часть изменчивости наблюдаемой переменной можно объяснить с помощью построенной модели. Значение с поправкой на степени свободы Adj R-squared также равно 0.87, что является очень хорошим результатом. 

Теперь выявим значимые переменные. Определенно к значимым ожно отнести переенные с x1 по x7, так как в столбце P > |t| для них указано нулевое значение. Переенная x8 значиам на уровне 1.2% и переменная x9 значима на уровне 10.3%.





#### 4. [1 point] Implement one of the elimination algorithms that were described in the Seminar_4 (Elimination by P-value, Forward elimination, Backward elimination), make conclusions.

In [ ]:
# your code here 
def get_fit(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 17)

    scaler = StandardScaler()
    X_train_scaled = sm.add_constant(scaler.fit_transform(X_train))
    X_test_scaled = sm.add_constant(scaler.transform(X_test))

    model = sm.OLS(y_train, X_train_scaled)
    fit = model.fit()
    prediction = fit.predict(X_test_scaled)

    return fit, mean_squared_error(y_test, prediction, squared = False)


def get_min(cur_drop, cur_data):
    min_rmse, min_drop, min_data = None, cur_drop, cur_data

    for i in range(cur_data.shape[1]):
        data_local = cur_data.drop(cur_data.columns[i - 1], axis = 1)
        drop_local, rmse_local = get_fit(data_local, y)
        if min_rmse is None or rmse_local <= min_rmse:
            min_rmse, min_drop, min_data = rmse_local, drop_local, data_local
    
    return min_rmse, min_drop, min_data


def backward_elimination(data, y):
    cur_data = data
    cur_drop, cur_rmse = get_fit(cur_data, y)
    min_rmse, min_drop, min_data = cur_rmse, cur_drop, cur_data

    while min_rmse <= cur_rmse:
        cur_rmse, cur_drop, cur_data = min_rmse, min_drop, min_data
        min_rmse, min_drop, min_data = get_min(cur_drop, cur_data)
  
    return cur_drop, cur_rmse

result, rmse = backward_elimination(X, y)

print(result.summary())
print("RMSE =", rmse)

                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.874
Method:                 Least Squares   F-statistic:                 3.742e+04
Date:                Sat, 08 Oct 2022   Prob (F-statistic):               0.00
Time:                        16:05:06   Log-Likelihood:            -3.7423e+05
No. Observations:               43152   AIC:                         7.485e+05
Df Residuals:                   43143   BIC:                         7.485e+05
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3928.6813      6.802    577.581      0.0

Мною был реализовн алгоритм backward elimination.
Ожидаемо, из списка переменных была "дропнута" x9, имевшая наибольшой значение P>|t|. После этого уровень значимости x8 улучшился.

В целом результаты модели не сильно изменились, так как она и до этого показывала хороший результат. Степеней свободы теперь 8.

#### 5. [1 point] Find the best (in terms of RMSE) $\alpha$ for Lasso regression using cross-validation with 4 folds. You must select values from range $[10^{-4}, 10^{3}]$.

In [ ]:
# your code here
model = GridSearchCV(Lasso(), [{"alpha": np.logspace(-4, 3)}], cv = 4, scoring = "neg_root_mean_squared_error")
model.fit(X_train_scaled, y_train)
print("Лучший результат с альфа =", model.best_params_["alpha"])

Лучший результат с альфа = 3.727593720314938


## Gradient descent

#### 6. [3.5 points] Implement a Ridge regression model for the MSE loss function, trained by gradient descent.

All calculations must be vectorized, and python loops can only be used for gradient descent iterations. As a stop criterion, you must use (simultaneously):

* checking for the Absolute-value norm of the weight difference on two adjacent iterations (for example, less than some small number of the order of $10^{-6}$, set by the `tolerance` parameter);
* reaching the maximum number of iterations (for example, 10000, set by the `max_iter` parameter).

You need to implement:

* Full gradient descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} Q(w_{k}).
$$

* Stochastic Gradient Descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} q_{i_{k}}(w_{k}).
$$

$\nabla_{w} q_{i_{k}}(w_{k}) \, $ is the estimate of the gradient over the batch of objects selected randomly.

* Momentum method:

$$
h_0 = 0, \\
h_{k + 1} = \alpha h_{k} + \eta_k \nabla_{w} Q(w_{k}), \\
w_{k + 1} = w_{k} - h_{k + 1}.
$$

* Adagrad method:

$$
G_0 = 0, \\
G_{k + 1} = G_{k} + (\nabla_{w} Q(w_{k+1}))^2, \\
w_{k + 1} = w_{k} - \eta * \frac{\nabla_{w} Q(w_{k+1})}{\sqrt{G_{k+1} + \epsilon}}.
$$



To make sure that the optimization process really converges, we will use the `loss_history` class attribute. After calling the `fit` method, it should contain the values of the loss function for all iterations, starting from the first one (before the first step on the anti-gradient).

You need to initialize the weights with a random vector from normal distribution. The following is a template class that needs to contain the code implementing all variations of the models.

In [ ]:
from sklearn.base import BaseEstimator
import math

'''
Пользовался этой статьей
https://machinelearningcompass.com/machine_learning_models/ridge_regression/

И вот этим видео
https://www.youtube.com/watch?v=C98SRCZfgkk

И вот этой статьей
https://d2l.ai/chapter_optimization/adagrad.html
'''

class MyRidge(BaseEstimator):
    def __init__(self, delta=1.0, gd_type='Momentum', tolerance=1e-4,
                 max_iter=1000, w0=None, eta=1e-2, alpha=1e-3, epsilon = 1):
        """
        gd_type: str
            'GradientDescent', 'StochasticDescent', 'Momentum', 'Adagrad'
        delta: float
            proportion of object in a batch (for stochastic GD)
        tolerance: float
            for stopping gradient descent
        max_iter: int
            maximum number of steps in gradient descent
        w0: np.array of shape (d)
            init weights
        eta: float
            learning rate
        alpha: float
            momentum coefficient
        reg_cf: float
            regularization coefficient
        epsilon: float
            numerical stability
        """
        
        self.delta = delta
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.loss_history = None
        self.epsilon = epsilon

        self.h = 0
        self.g = 0
        self.gd_functions = {'GradientDescent': self.gradient_descent,
                             'StochasticDescent': self.stochastic_descent,
                             'Momentum': self.momentum,
                             'Adagrad': self.adagrad}
    
    def fit(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: self
        """
        self.loss_history = []
        self.w = self.w0

        cur_iteration = 0
        while cur_iteration < self.max_iter and (cur_iteration == 0 or self.tolerance < self.loss_history[-1]):
            weight = self.gd_functions[self.gd_type](X, y)
            self.w, weight = weight, self.w
            self.loss_history.append(self.calc_loss(X, y))
            cur_iteration += 1

        return self
    
    def predict(self, X):
        return X.dot(np.transpose(self.w))
    
    def calc_gradient(self, X, y):
        return 2 * np.dot(X.T, self.predict(X) - y) / y.shape[0]

    def calc_loss(self, X, y):
        rss = np.sum(np.power(self.predict(X) - y, 2))
        penalty = np.sum(np.power(self.w, 2))
        return np.mean(rss + penalty)
    
    # -----------------------------------------------------------------------
    
    def gradient_descent(self, X, y):
        return self.w - self.eta * self.calc_gradient(X, y)
        
    def stochastic_descent(self, X, y):
        index = np.random.randint(0, len(X), math.floor(self.delta * len(X)))
        return self.w - self.eta * self.calc_gradient(X[index, :], y.to_numpy()[index])
        
    def momentum(self, X, y):
        self.h = self.alpha * self.h + self.eta * self.calc_gradient(X, y)
        return self.w - self.h

    def adagrad(self, X, y):
        self.g = self.g + np.power(self.calc_gradient(X, y), 2)
        return self.w - self.eta * self.calc_gradient(X, y) / np.sqrt(self.g + self.epsilon)


#### 7. [1 points] Train and validate "hand-written" models on the same data, and compare the quality with the Sklearn or StatsModels methods. Investigate the effect of the `max_iter` and `alpha` parameters on the optimization process. Is it consistent with your expectations?

In [ ]:
# your code here

def model_results(model, name):
    trained = model.fit(X_train_scaled, y_train)
    prediction = trained.predict(X_test_scaled)
    rmse = mean_squared_error(y_test, prediction, squared = False)
    r2 = r2_score(y_test, prediction)
    print(f"[{name}] RMSE = {rmse}, r2 = {r2}")

random_w0 = np.random.rand(X_train_scaled[0].size)
sk_model = sklearn.linear_model.Ridge()
my_model = MyRidge(gd_type = 'GradientDescent', w0 = random_w0)
model_results(sk_model, "SK")
model_results(my_model, "MY")
print()

max_iters = [100, 500, 1000, 2000, 5000]
for cur_iter in max_iters:
    iter_model = MyRidge(gd_type = 'GradientDescent', w0 = random_w0, max_iter = cur_iter)
    model_results(iter_model, f"ITER = {cur_iter}")
print()


alphas = [0, 0.01, 0.05, 0.1, 0.5, 0.9, 0.99, 0.995, 0.999, 1]
for cur_alpha in alphas:
    alpha_model = MyRidge(gd_type = 'Momentum', w0 = random_w0, alpha = cur_alpha)
    model_results(alpha_model, f"ALPHA = {cur_alpha}")
print()

[SK] RMSE = 1448.2313052658506, r2 = 0.8704814590613568
[MY] RMSE = 1499.6082828068595, r2 = 0.8611289426336216

[ITER = 100] RMSE = 1753.0857600372515, r2 = 0.8102147722448776
[ITER = 500] RMSE = 1563.178759960914, r2 = 0.84910551334503
[ITER = 1000] RMSE = 1499.6082828068595, r2 = 0.8611289426336216
[ITER = 2000] RMSE = 1459.7729380638327, r2 = 0.8684088453348653
[ITER = 5000] RMSE = 1448.1444649338687, r2 = 0.870496991243042

[ALPHA = 0] RMSE = 1499.6082828068595, r2 = 0.8611289426336216
[ALPHA = 0.01] RMSE = 1498.7927116436017, r2 = 0.8612799533111883
[ALPHA = 0.05] RMSE = 1495.505670176956, r2 = 0.8618877472148415
[ALPHA = 0.1] RMSE = 1491.350448928271, r2 = 0.8626541631694599
[ALPHA = 0.5] RMSE = 1459.7510047372784, r2 = 0.8684127996622933
[ALPHA = 0.9] RMSE = 1447.81555362485, r2 = 0.8705558115720164
[ALPHA = 0.99] RMSE = 1448.5553492163947, r2 = 0.8704234926371899
[ALPHA = 0.995] RMSE = 1482.938907533331, r2 = 0.8641991147917698
[ALPHA = 0.999] RMSE = 2967.143594727026, r2 = 0.

1) Результаты мною реализовнной Ridge модели слегка хуже аналогичной в Sklearn
Мой RMSE = 1499, sklearn rmse = 1448
Мой r2 = 0.86, sklearn r2 = 0.87

2) Кол-во итераций предсказуеом влияет на результат. Чем больше итераций, тем он лучше (rmse уменьшается, r2 растет). Кстиати, при 5000 итераций, мой класс показывает лучше результаты, чем Sklearn.

3) При росте параматера альфа (тип = momentum) от 0 до 0.9 результаты улучшаются. Далее при альфа от 0.9 до 0.99 медленно начинают ухудашаться и после 0.99 до 1 происходит резкое падение r2 и огромный рост RMSE. Это объясняется тем, что чем сильнее альфа приблежается к единице, тем больше начинают влиять на конечный результат предудыщие значения self.h.

#### 8. [1 points] Plot graphs (on the same picture) of the dependence of the loss function value on the iteration number for Full GD, SGD, Momentum and Adagrad. Draw conclusions about the rate of convergence of various modifications of gradient descent.

Don't forget about what *beautiful* graphics should look like!

In [ ]:
# your code here

types = {'GradientDescent': "blue",
        'StochasticDescent': "green",
        'Momentum': "red",
        'Adagrad': "yellow"}
types.pop("Adagrad")

plt.figure(figsize=(30, 10))
for cur_type, color in types.items():
    model = MyRidge(gd_type = cur_type, w0 = random_w0, delta = 0.2, alpha = 0.3, epsilon = 0.000001)
    trained = model.fit(X_train_scaled, y_train)
    plt.plot(trained.loss_history[:150], label = cur_type, color = color)
plt.legend()
plt.show()

Общий характер графиков похож. Все методы примерно за 40-50 итерация снижают потери до 0.2 * 10 ^ 12 единиц. При этом gradient descent и stochastic descent очень похожи. Momentum показывает слегка большую эффективность, чем два предыдущих метода, при этом плавность данной кривой напрямую зависит от параметра alpha. Если начать его изменять, то график начнет прыгать и вырисовывать "горы".